In [7]:
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
import gensim as gs
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
import lda
from load import all_subreddits_data, tv_subreddits_data

In [8]:
# data is of the form {class_label: list of documents}
all_data = all_subreddits_data()
# tv_data = tv_subreddits_data()

Error: line contains NULL byte

In [36]:
all_data.keys()
# all_data2 = {'buildapc': all_data['buildapc'], 'anime': all_data['anime']}

['buildapc',
 'AskWomen',
 'aww',
 'amiibo',
 'AdviceAnimals',
 '2007scape',
 'anime',
 'asoiaf',
 'AskMen',
 'CasualConversation']

['buildapc',
 'AskWomen',
 'aww',
 'amiibo',
 'AdviceAnimals',
 '2007scape',
 'anime',
 'asoiaf',
 'AskMen',
 'CasualConversation']

In [7]:
def tt_split(data, test_size=.1):
    """Splits a dictionary {class_label: list of documents}"""
    """into two dictionaries of the same shape"""
    train_data = {}; test_data = {}
    for label, docs in data.iteritems():
        train, test = train_test_split(docs, test_size=test_size)
        train_data[label] = train
        test_data[label] = test
    return train_data, test_data
        
        

In [8]:
def tokens_to_vocab(class_tokens):
    """{class_label : list of tokenized documents} -> vocab"""
    vocab = set([])
    for _class, tokenized_docs in class_tokens.iteritems():
        for d in tokenized_docs:
            vocab = vocab.union(set(d))
    return {word: i for i, word in enumerate(vocab)}
        

def word_tokenize_doc(doc):
    """Word tokenize a single document"""
    to_remove = set(['http', 'faq', 'https', 'amp','source', 'deletion', 'sfw',
              'nsfw', 'gt', 'gon', 'na', 'delete', 'comment', 'profile'])
    def _filter(w):
        return all([w.isalnum(), w not in stopwords.words('english'), w not in to_remove])
    tokens = word_tokenize(doc)
    tokens = filter(_filter, tokens)
    return tokens

def tokenize_all_words(data):
    """basic get_tokens method"""
    """{class_label: list of documents} ->""" 
    """{class_label : list of tokenized documents}"""
    for c, docs in data.iteritems():
        data[c] = map(word_tokenize_doc, docs)
    return data

# test_data = {
#     'c1': ['aa bb c', 'b d'],
#     'c2': ['d e f', 'e g']
# }
# test_tokens = tokenize_all_words(test_data)
# test_vocab = tokens_to_vocab(test_tokens)


In [43]:
# def get_lda_topics(train_tokens, vocab, n_topics=30):
#     """{class_label: list of tokenized docs} ->"""
#     """{class_label: list of topic distributions from LDA"""
#     all_class_tokens = train_tokens.values()
#     flattened_classes = [item for sublist in all_class_tokens for item in sublist]
#     flattened_documents = [item for sublist in flattened_classes for item in sublist]
#     vectorizer = CountVectorizer(min_df=2, vocabulary=vocab)
#     X = vectorizer.fit_transform(flattened_documents)
#     lda = LatentDirichletAllocation(n_topics=n_topics)
#     X_new = lda.fit_transform(X.toarray())
#     return X_new

def get_hlda_models(train_tokens, vocab, n_topics=40):
    models = {}
    dictionary = gs.corpora.Dictionary(map(lambda x: [x], vocab.keys()))
    for label, docs in train_tokens.items():
        corpus = [dictionary.doc2bow(d) for d in docs]
        models[label] = gs.models.HdpModel(corpus, dictionary, T=n_topics)
    return models, dictionary

def get_lda_models(train_tokens, vocab, n_topics=40):
    all_models = {}
    def fit_model((label, docs)):
        model = lda.LDA(n_topics=n_topics, n_iter=1500)
        vectorizer = CountVectorizer(min_df=2, vocabulary = vocab, stop_words=None)
        X = vectorizer.fit_transform(map(lambda s: ' '.join(s), docs))
        model.fit(X)
        all_models[label] = model
        print 'done fitting for ', label
    map(fit_model, train_tokens.items())
    return all_models


def hlda_pred(models, dictionary, doc):
    corpus = [dictionary.doc2bow(word_tokenize_doc(doc))]
    label_score = []
    for label, hdp in models.iteritems():
        label_score.append((label, hdp.evaluate_test_corpus(corpus)))
    return max(label_score, key = lambda x:x[1])[0]

def lda_pred(models, vocab, doc):
    """Get a class prediction for a document """
    tokenized = word_tokenize_doc(doc)
    vectorizer = CountVectorizer(min_df=1, vocabulary = vocab, stop_words=None)
    X = vectorizer.fit_transform([' '.join(tokenized)])
    label_score = []
    for label, model in models.iteritems():
        n_topics = len(model.components_)
        topic_dist = model.transform(X)
        log_likelihood = 0
        for token in tokenized:
            if token in vocab:
                max_likelihood = -1 * 10 ** 8
                for topic in range(n_topics):
                    ll = np.log(model.components_[topic][vocab[token]]) + np.log(topic_dist[0][topic])
                    max_likelihood = max_likelihood if max_likelihood > ll else ll
                log_likelihood += max_likelihood
        label_score.append((label, log_likelihood))
    return max(label_score, key = lambda x:x[1])[0]
# get_lda_topics(test_tokens, test_vocab)

In [37]:
# def run_model(data, get_tokens=tokenize_all_words,
#               get_models = get_lda_models):
#     """Vectorizes, topic models, classifies, and returns score"""
data = all_data
get_tokens = tokenize_all_words
get_models = get_lda_models

train, test = tt_split(data)
print 'done splitting'
train_tokens = get_tokens(train)
print 'done tokenizing'
vocab = tokens_to_vocab(train_tokens)



done splitting
done tokenizingdone splitting
done tokenizing



In [39]:
hlda_models, dictionary = get_hlda_models(train_tokens, vocab)

In [13]:
# print pred(Out[30], vocab, 'qi')
# print pred(Out[30], vocab, 'gold')

In [40]:
correct = 0
total = 0
for label, docset in test.iteritems():
    for doc in docset:
        total += 1
        if hlda_pred(hlda_models, dictionary, doc) == label:
            correct += 1
print correct, total

1386 1706
1386 1706


In [ ]:
lda_models = get_lda_models(train_tokens, vocab)


In [ ]:
correct = 0
total = 0
for label, docset in test.iteritems():
    for doc in docset:
        total += 1
        if lda_pred(lda_models, vocab, doc) == label:
            correct += 1
print correct, total

In [1]:
def get_lb(delta, D, L, V):
    scale = 1./np.sqrt(D*delta)
    factor = np.sqrt(2*L**(-2) + 2*V**(-2))
    return scale*factor

def get_max_token_id(token_lists):
    return max([x[0] for sublist in token_lists for x in sublist])

def expand_token(token, V):
    vec = np.array([0]*V)
    vec[token[0]] = 1
    return vec

def get_M1(token_lists, V):
    tls_np = np.array(token_lists)
    D = tls_np.shape[0]
    L = tls_np.shape[1]
    M1 = np.array([0]*V)
    for d in range(D):
        for l in range(L):
            M1 += expand_token(tls_np[d,l,:], V)
    return M1 / (D*L)

def get_M2(token_lists, V, M1, alphas):
    alpha0 = alphas.sum()
    tls_np = np.array(token_lists)
    sum_of_outers = np.array(0, shape=(V,V))
    for d in range(D):
        for l1 in range(L):
            for l2 in range(L):
                if l1 != l2:
                    sum_of_outers += np.outer(expand_token(tls_np[d, l1, :], V), expand_token(tls_np[d, l2, :], V))
    M1_outer = alpha0/(alpha0+1)*np.outer(M1, M1)
    return sum_of_outers - M1_outer

def get_sv1(M2):
    return np.linalg.svd(M2, compute_uv=False)[0]

def deltaprime(K, d2, d3, V, beta):
    numer = np.log(K/d3)*K*(beta + 2*np.log(K/d2))**2
    denom = V*beta
    return np.sqrt(numer/denom)

def get_sigbar1(K, d1, d2, d3, V, beta, alphas):
    a_max = max(alphas)
    a_0 = alphas.sum()
    a_frac = a_max/(a_0*(a_0 + 1))
    numer = (1 + deltaprime(K, d2, d3, V, beta))*V*(beta + K*beta**2)
    denom = (V*beta - np.sqrt(2*V*beta*np.log(K/d1)))**2
    return a_frac*numer/denom

def get_ub(K_guess, d1, d2, d3, V, beta, alphas):
    func = lambda K : get_sigbar1(K, d1, d2, d3, V, beta, alphas)
    try:
        return scipy.optimize.fsolve(func, K_guess)
    except:
        logger.warning("Unable to solve for K. Setting upper bound to 1000.")
        return 1000